In [1]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
import datetime as dt
import string

#visualizing results
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("white")
sns.set_context("talk")

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 15000)
pd.set_option('display.max_colwidth', -1)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn import metrics
from sklearn.manifold import TSNE

from sklearn.cluster import KMeans

import scipy.cluster.hierarchy as shc

/Users/abbieschindler/opt/anaconda3/envs/analysis/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


### Get data and gb day and substance

In [3]:
path = 'data_final_clean_3s_221019.csv'
    
data = pd.read_csv(path)
data = pd.DataFrame(data = data)
print(data.shape)
data.head()

(65191, 51)


,Unnamed: 0,Unnamed: 0.1,Round,Build,Startdate,Timepoint,Cage,IdRFID,Animal,ARF_Cage,Injury,Sex,Cage_N,weight_pre,weight_post,weight_diff,notes,index,VDM_file,Cage_VDM,VDM_Time,variable,value,VDM_datetime,VDM_timestamp,VDM_date,VDM_day,VDM_time,VDM_hour,VDM_min,VDM_second,ld_seg,VDM_drink,Cage_VDM_update,RFID_eventDuration,RFID_assign_id_pre,RFID_assign_var_pre,RFID_datetime_start_pre,RFID_datetime_end_pre,ld_cycle,ld_day,VDM_RFID_timediff,VDM_RFID_timediff_bool,day_count,min_count,sec_count,hour_count_running,min_count_running,sec_count_running,substance,value_wt
0,0,0,poly,six_four,22.10.10,na,1,041AC165FE,2137.0,NaN,na,male,4,29.7,NaN,NaN,NaN,2,w_e10_f20_w_e5_f5_mf_221010.csv,2,10/10/22 16:17:51,drink_1,0.021,2022-10-10 16:17:51,1.665419e+09,2022-10-10,10,16:17:51,16,17,51,light,VDM_EtOH05,1.0,22.395,041AC165FE,etoh_05,2022-10-10 16:17:46.580035200,2022-10-10 16:18:08.975035200,light,1,0.0,same,1,977,58671,16,977,58611,EtOH05,0.707071
1,1,1,poly,six_four,22.10.10,na,1,041AC165FE,2137.0,NaN,na,male,4,29.7,NaN,NaN,NaN,3,w_e10_f20_w_e5_f5_mf_221010.csv,2,10/10/22 16:17:54,drink_1,0.021,2022-10-10 16:17:54,1.665419e+09,2022-10-10,10,16:17:54,16,17,54,light,VDM_EtOH05,1.0,22.395,041AC165FE,etoh_05,2022-10-10 16:17:46.580035200,2022-10-10 16:18:08.975035200,light,1,0.0,same,1,977,58674,16,977,58614,EtOH05,0.707071
2,2,2,poly,six_four,22.10.10,na,1,041AC165FE,2137.0,NaN,na,male,4,29.7,NaN,NaN,NaN,4,w_e10_f20_w_e5_f5_mf_221010.csv,2,10/10/22 16:17:57,drink_1,0.021,2022-10-10 16:17:57,1.665419e+09,2022-10-10,10,16:17:57,16,17,57,light,VDM_EtOH05,1.0,22.395,041AC165FE,etoh_05,2022-10-10 16:17:46.580035200,2022-10-10 16:18:08.975035200,light,1,0.0,same,1,977,58677,16,977,58617,EtOH05,0.707071
3,3,3,poly,six_four,22.10.10,na,1,041AC165FE,2137.0,NaN,na,male,4,29.7,NaN,NaN,NaN,5,w_e10_f20_w_e5_f5_mf_221010.csv,2,10/10/22 16:17:59,drink_1,0.021,2022-10-10 16:17:59,1.665419e+09,2022-10-10,10,16:17:59,16,17,59,light,VDM_EtOH05,1.0,22.395,041AC165FE,etoh_05,2022-10-10 16:17:46.580035200,2022-10-10 16:18:08.975035200,light,1,0.0,same,1,977,58679,16,977,58619,EtOH05,0.707071
4,4,4,poly,six_four,22.10.10,na,1,041AC165FE,2137.0,NaN,na,male,4,29.7,NaN,NaN,NaN,15,w_e10_f20_w_e5_f5_mf_221010.csv,2,10/10/22 17:28:38,drink_1,0.021,2022-10-10 17:28:38,1.665423e+09,2022-10-10,10,17:28:38,17,28,38,light,VDM_EtOH05,1.0,19.446,041AC165FE,etoh_05,2022-10-10 17:28:30.700963200,2022-10-10 17:28:50.146963200,light,1,0.0,same,1,1048,62918,17,1048,62858,EtOH05,0.707071


In [ ]:
#create new column with 6 four-hour time segments

def assign_timeseg(time):
    if (time >= 6) & (time < 10):
        return 1
    if (time >= 10) & (time < 14):
        return 2
    if (time >= 14) & (time < 18):
        return 3
    if (time >= 18) & (time < 22):
        return 4
    if (time >= 22) | (time < 2):
        return 5
    if (time >= 2) & (time < 6):
        return 6
    
data['six_seg'] = data['VDM_hour'].apply(assign_timeseg)

print(data.shape)
data.head()

In [ ]:
data_gb_daysum = orig_data.groupby(['Sex', 'Cage', 'IdRFID', 
                                   'substance'])['value',].sum().reset_index()

data_gb_daysum.head()

In [ ]:
unstack_flavor_params = ['Sex', 'Cage', 'IdRFID', 'substance', 'value']

unstack_flavor = data_gb_daysum[unstack_flavor_params].set_index(['Sex', 'Cage', 'IdRFID', 'substance']).unstack(-1).reset_index()
unstack_flavor.columns = ['Sex', 'Cage', 'IdRFID', 'EtOH05', 'EtOH10', 'Fent05', 'Fent20', 'water']

unstack_flavor.head()

In [ ]:
unstack_flavor = unstack_flavor.fillna(0)

unstack_flavor.head()

In [ ]:
unstack_flavor_day = unstack_flavor.set_index(['Sex', 'Cage', 'IdRFID', 'day_count',]).unstack(-1).reset_index()

unstack_flavor_day.columns = ['Sex', 'Cage', 'IdRFID', 
                              'EtOH05_1', 'EtOH05_2', 'EtOH05_3', 'EtOH05_4', 'EtOH05_5', 
                              'EtOH05_6', 'EtOH05_7', 'EtOH05_8', 'EtOH05_9',
                              
                              'EtOH10_1', 'EtOH10_2', 'EtOH10_3', 'EtOH10_4', 'EtOH10_5', 
                              'EtOH10_6', 'EtOH10_7', 'EtOH10_8', 'EtOH10_9',
                              
                              'Fent05_1', 'Fent05_2', 'Fent05_3', 'Fent05_4', 'Fent05_5', 
                              'Fent05_6', 'Fent05_7', 'Fent05_8', 'Fent05_9',
                              
                              'Fent20_1', 'Fent20_2', 'Fent20_3', 'Fent20_4', 'Fent20_5', 
                              'Fent20_6', 'Fent20_7', 'Fent20_8', 'Fent20_9',
                              
                              'water_1', 'water_2', 'water_3', 'water_4', 'water_5', 
                              'water_6', 'water_7', 'water_8', 'water_9',]

print(unstack_flavor_day.shape)
unstack_flavor_day.head()

In [ ]:
unstack_flavor_day = unstack_flavor_day.fillna(0)

unstack_flavor_day.head()

### Cluster

In [ ]:
data_cluster = unstack_flavor_day[['EtOH05_1', 'EtOH05_2', 'EtOH05_3', 'EtOH05_4', 'EtOH05_5', 
                              'EtOH05_6', 'EtOH05_7', 'EtOH05_8', 'EtOH05_9',
                              
                              'EtOH10_1', 'EtOH10_2', 'EtOH10_3', 'EtOH10_4', 'EtOH10_5', 
                              'EtOH10_6', 'EtOH10_7', 'EtOH10_8', 'EtOH10_9',
                              
                              'Fent05_1', 'Fent05_2', 'Fent05_3', 'Fent05_4', 'Fent05_5', 
                              'Fent05_6', 'Fent05_7', 'Fent05_8', 'Fent05_9',
                              
                              'Fent20_1', 'Fent20_2', 'Fent20_3', 'Fent20_4', 'Fent20_5', 
                              'Fent20_6', 'Fent20_7', 'Fent20_8', 'Fent20_9',
                              
                              'water_1', 'water_2', 'water_3', 'water_4', 'water_5', 
                              'water_6', 'water_7', 'water_8', 'water_9',]]

# center and scale the data
scaler = StandardScaler()
#scaler = RobustScaler()

data_cluster_scaled = scaler.fit_transform(data_cluster)

In [ ]:
data_cluster = unstack_flavor[['EtOH05', 'EtOH10', 'Fent05', 'Fent20', 'water']]

# center and scale the data
scaler = StandardScaler()
#scaler = RobustScaler()

data_cluster_scaled = scaler.fit_transform(data_cluster)

In [ ]:
data = data_cluster_scaled

#pick cluster number based on silhouette coefficient
k_range = range(2,9)

base_scores = []
sil_scores = []
db_scores = []
ch_scores = []
mse_scores = []

for k in k_range:
    km_ss = KMeans(n_clusters=k, random_state=39)
    km_ss.fit(data)
    
    sil_scores.append(metrics.silhouette_score(data, km_ss.labels_))
    db_scores.append(metrics.davies_bouldin_score(data, km_ss.labels_))
    ch_scores.append(metrics.calinski_harabasz_score(data, km_ss.labels_))
    mse_scores.append(km_ss.inertia_)
    

# plot the results
plt.plot(k_range, sil_scores)
plt.title('K means clustering')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')
plt.show()

# plot the results
plt.plot(k_range, db_scores)
plt.title('K means clustering')
plt.xlabel('Number of clusters')
plt.ylabel('Davies Bouldin Score')
plt.show()

# plot the results
plt.plot(k_range, ch_scores)
plt.title('K means clustering')
plt.xlabel('Number of clusters')
plt.ylabel('Calinski Harabasz Score')
plt.show()
    
# plot the results
plt.plot(k_range, mse_scores)
plt.title('K means clustering')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
#viz dendrogram to find if three clusters makes sense
plt.figure(figsize=(10, 7))  
plt.title("Dendrogram")  
plt.ylabel("Distance (dissimilarity)")
plt.xlabel("Mice")
dend = shc.dendrogram(shc.linkage(data_cluster_scaled, method='ward'), 
                      distance_sort='ascending', 
                      show_leaf_counts=True, leaf_font_size=8)

In [ ]:
params = ['EtOH05', 'EtOH10', 'Fent05', 'Fent20', 'water']

lut = dict(zip(unstack_flavor['Sex'].unique(), "bgyr"))
row_colors = unstack_flavor['Sex'].map(lut)
plt.figure(figsize=(20,15))

g = sns.clustermap(unstack_flavor[params], row_colors=row_colors,
                 metric="euclidean", z_score=1, method="ward",
               vmin=-1, vmax=1, center=0, cmap = 'coolwarm', 
               square=True, linewidths=.5, cbar_kws={"shrink": .5}, figsize=(15,15))
plt.show()

In [ ]:
#choose k=3 clusters and fit data
km_3 = KMeans(n_clusters=3,random_state=99)
km_3.fit(data_cluster_scaled)

unstack_flavor['kmeans_cluster'] = ["cluster_" + str(label) for label in km_3.labels_ ]
print(unstack_flavor.shape)
unstack_flavor.head(1)

### Viz

#### what can we see in the raw data

In [ ]:
data_gb_sum = orig_data.groupby(['Sex', 'Cage', 'IdRFID', 'substance'])['value',].sum().reset_index()

sns.catplot(x='Sex', y='value', data=data_gb_sum,  kind='bar', 
            ci=68, height=5, aspect=1, hue='substance',
           )

plt.show()

In [ ]:
groupby = data_gb_sum.pivot('IdRFID', 'substance', "value")
plt.figure(figsize=(10,10))
ax = sns.heatmap(groupby, cmap="Reds", vmin=0, vmax=15, cbar_kws={'label': 'Total intake (ml)'})
    #plt.xlabel('Bottle')
plt.show()

#### do we see something more with the clustering

In [ ]:
data_gb_sum = data_gb_sum.merge(unstack_flavor[['IdRFID', 'kmeans_cluster']], on='IdRFID')
print(data_gb_sum.shape)
data_gb_sum.head()

In [ ]:
order = ['cluster_0', 'cluster_1', 'cluster_2' ]
sns.catplot(x='kmeans_cluster', y='value', kind='bar', ci=68, data=data_gb_sum, 
            height=5, aspect=2, hue='substance', order=order, col='Sex', col_wrap=1,
            sharex=False,
           )
plt.show()